In [1]:
import numpy as np
import tensorflow as tf
import test_utils_models

2024-04-30 19:49:50.563686: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 19:49:51.490291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def custom_loss_function(y_true, y_pred, beta=0.1, financial_penalty_factor=0.01):
    decoder_output = y_pred[:, :, :-1]
    end_decision = y_pred[:, :, -1]  

    y_true_cum = tf.cumsum(y_true, axis=1)
    decoder_output = tf.cumsum(decoder_output, axis=1)

    print("Y_TRUE ", y_true)

    mse_loss = tf.keras.losses.mean_squared_error(y_true_cum, decoder_output)
    print("MSE LOSS", mse_loss)
    # mse_loss = tf.reduce_mean(mse_loss)
    # print("MSE LOSS_REDUCED", mse_loss)

    sequence_length = tf.shape(decoder_output)[1]
    # cumulative_stop_decision = tf.cumsum(end_decision, axis=1)
    print("CUMULATIVE_STOP_DECISION", end_decision)
    weights = tf.nn.relu(1.0 - beta * end_decision)

    print("WEIGHTS", weights)
    
    weighted_errors = mse_loss * weights
    print("WEIGHTED ERROR", weighted_errors)

    # Reward for longer predictions
    reverse_cumulative_profits = tf.cumsum(tf.abs(y_true[:, ::-1]), axis=1)[:, ::-1]
    print("Reverse Cumulative Profits shape ", reverse_cumulative_profits)
    weights_financial = tf.exp(tf.cast(reverse_cumulative_profits, tf.float64) / 10 + 1e-10)
    print("financial Weights shape ", weights_financial)

    financial_penalty = financial_penalty_factor * tf.reduce_sum(weights_financial * tf.expand_dims(1 - tf.cast(end_decision, tf.float64), axis=1), axis=1)
    print("Financial Penalty shape ", financial_penalty)
    # Normalization
    mse_norm = mse_loss / tf.reduce_mean(mse_loss)
    financial_penalty_norm = financial_penalty / tf.reduce_mean(financial_penalty)
    print("MSE NORM shape ", mse_norm)
    print("Financial Penalty NORM shape ", financial_penalty_norm)
    

    # Total loss calculation
    total_loss = tf.reduce_mean(tf.cast(weighted_errors * mse_norm, tf.float32)) + tf.reduce_mean(tf.cast(financial_penalty_norm, tf.float32))

    return total_loss

In [3]:
# Create y_fake with shape (1, 20, 1) and positive values
y_fake = np.random.uniform(0, 1, size=(1, 20, 1))
# y_fake = np.concatenate([y_fake,np.random.uniform(0, 1, size=(1, 20, 1))], axis=0)

y_fake_true = np.random.uniform(0, 1, size=(1, 20, 1))  
# y_fake_true = np.concatenate([y_fake_true,np.random.uniform(0, 1, size=(1, 20, 1))], axis=0)

# Create end_decision_fake with shape (1, 20, 1) and values that start small and progressively grow
end_decision_fake = np.linspace(1, 0, 20).reshape(1, 20, 1)
# end_decision_fake = np.concatenate([end_decision_fake, np.linspace(0, .12, 20).reshape(1, 20, 1)], axis=0)

In [4]:
print("Y_FAKE", y_fake.shape)
print("Y_FAKE_TRUE", y_fake_true.shape)
print("END_DECISION_FAKE", end_decision_fake.shape)

Y_FAKE (1, 20, 1)
Y_FAKE_TRUE (1, 20, 1)
END_DECISION_FAKE (1, 20, 1)


In [5]:
beta = 1
financial_penalty_factor = 100

In [6]:
custom_loss_function(y_fake_true, np.concatenate([y_fake, end_decision_fake], axis=-1), beta, financial_penalty_factor)

Y_TRUE  [[[0.79945313]
  [0.37449299]
  [0.48442149]
  [0.48109395]
  [0.35068947]
  [0.3147582 ]
  [0.99637336]
  [0.29540498]
  [0.74466513]
  [0.83708861]
  [0.65572253]
  [0.18426676]
  [0.85817277]
  [0.94875308]
  [0.7373784 ]
  [0.1038553 ]
  [0.34686563]
  [0.11367122]
  [0.70306242]
  [0.51155827]]]
MSE LOSS tf.Tensor(
[[0.0596287  0.27503417 0.28415166 0.51605309 0.380905   0.07181017
  0.18095166 0.3805613  0.30255876 0.7758133  1.80985088 0.2961877
  0.65155992 0.83041544 0.91475427 0.7330044  1.24602678 0.26963034
  0.2710197  0.09295616]], shape=(1, 20), dtype=float64)
CUMULATIVE_STOP_DECISION [[1.         0.94736842 0.89473684 0.84210526 0.78947368 0.73684211
  0.68421053 0.63157895 0.57894737 0.52631579 0.47368421 0.42105263
  0.36842105 0.31578947 0.26315789 0.21052632 0.15789474 0.10526316
  0.05263158 0.        ]]
WEIGHTS tf.Tensor(
[[0.         0.05263158 0.10526316 0.15789474 0.21052632 0.26315789
  0.31578947 0.36842105 0.42105263 0.47368421 0.52631579 0.57894737


2024-04-30 19:49:52.843430: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 19:49:52.887599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 19:49:52.887865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

<tf.Tensor: shape=(), dtype=float32, numpy=1.5298294>

In [7]:
def custom_loss(y_true, y_pred_full, financial_penalty = .5):
    # Split predictions and decision signals
    y_pred = y_pred_full[:, :, :-1]  # Predicted day-over-day percent changes
    decision = y_pred_full[:, :, -1:]  # Decision output (already sigmoid)
    print("Decision", decision) 

    # Calculate cumulative sums
    # In real trading, we'd consider each day's change as an independent event where the model decides to 'buy' (predict positive) or 'sell' (predict negative).
    y_true_cum = tf.cumsum(y_true, axis=1)
    y_pred_cum = tf.cumsum(y_pred, axis=1)
    print("Y_TRUE_CUM", y_true_cum)
    print("Y_PRED_CUM", y_pred_cum)

    # Calculate the profit/loss based on the trading strategy:
    # Profit/loss is computed as the product of the prediction and the actual change.
    # This mimics the gain or loss from going 'long' or 'short' based on the sign and magnitude of the prediction.
    # profit_loss = -(y_pred_cum * y_true_cum) / 100  # Negative because we minimize loss in training
    cumulative_profit_loss = y_pred_cum * y_true_cum

    print("cumulative profit loss ", cumulative_profit_loss)

    avg_profit_loss_per_day = tf.reduce_sum(cumulative_profit_loss, axis=1) / tf.cast(tf.shape(y_true_cum)[1], tf.float64)
    avg_profit_loss_per_day = tf.expand_dims(avg_profit_loss_per_day, -1)

    profit_loss = -avg_profit_loss_per_day
    print("PROFIT_LOSS", profit_loss)
    # Regulate profit/loss by the decision to act
    regulated_loss = decision * profit_loss
    print("REGULATED_LOSS", regulated_loss)

    # Penalty for not acting when there is significant market movement
    y_true_reverse_cum = tf.reverse(tf.cumsum(tf.reverse(y_true, [1]), axis=1), [1])
    abstain_penalty = financial_penalty * (1 - decision) * y_true_reverse_cum  # Apply penalty based on reverse cumulative movements missed
    print("ABSTAIN_PENALTY", abstain_penalty)

    prediction_error = tf.square(y_true_cum - y_pred_cum)   # Penalize the model for incorrect predictions

    # Combine into a total loss
    total_loss = tf.reduce_mean(regulated_loss + abstain_penalty + prediction_error)

    return total_loss


In [8]:
custom_loss(y_fake_true, np.concatenate([y_fake, end_decision_fake], axis=-1))

Decision [[[1.        ]
  [0.94736842]
  [0.89473684]
  [0.84210526]
  [0.78947368]
  [0.73684211]
  [0.68421053]
  [0.63157895]
  [0.57894737]
  [0.52631579]
  [0.47368421]
  [0.42105263]
  [0.36842105]
  [0.31578947]
  [0.26315789]
  [0.21052632]
  [0.15789474]
  [0.10526316]
  [0.05263158]
  [0.        ]]]
Y_TRUE_CUM tf.Tensor(
[[[ 0.79945313]
  [ 1.17394612]
  [ 1.65836761]
  [ 2.13946156]
  [ 2.49015103]
  [ 2.80490923]
  [ 3.80128259]
  [ 4.09668756]
  [ 4.84135269]
  [ 5.6784413 ]
  [ 6.33416383]
  [ 6.51843059]
  [ 7.37660336]
  [ 8.32535644]
  [ 9.06273484]
  [ 9.16659014]
  [ 9.51345577]
  [ 9.62712699]
  [10.3301894 ]
  [10.84174768]]], shape=(1, 20, 1), dtype=float64)
Y_PRED_CUM tf.Tensor(
[[[ 0.55526325]
  [ 0.64950912]
  [ 1.12530884]
  [ 1.42109321]
  [ 1.87297601]
  [ 2.53693504]
  [ 3.37589845]
  [ 3.47979106]
  [ 4.29129928]
  [ 4.79763865]
  [ 4.98885685]
  [ 5.9741993 ]
  [ 6.56941075]
  [ 7.41408511]
  [ 8.10630697]
  [ 8.31043338]
  [ 8.39720007]
  [ 9.10786757]
 

<tf.Tensor: shape=(), dtype=float64, numpy=-18.750961722838415>

In [13]:
def custom_loss_updated(y_true, y_pred_full, profit_multiplier = 1 ,financial_penalty = 0, error_weight = 1):
    # Split predictions and decision signals
    y_pred = y_pred_full[:, :, :-1]  # Predicted day-over-day percent changes
    decision = y_pred_full[:, :, -1:]  # Decision output (already sigmoid)
    print("Decision", decision) 

    # Calculate cumulative sums
    # In real trading, we'd consider each day's change as an independent event where the model decides to 'buy' (predict positive) or 'sell' (predict negative).
    y_true_cum = tf.cumsum(y_true, axis=1)
    y_pred_cum = tf.cumsum(y_pred, axis=1)
    print("Y_TRUE_CUM", y_true_cum)
    print("Y_PRED_CUM", y_pred_cum)

    profit = tf.zeros_like(y_true_cum)
    day_numbers = tf.cast(tf.range(1, tf.shape(y_true)[1] + 1), dtype=tf.float64)
    day_numbers = tf.reshape(day_numbers, (1, -1, 1))
    reverse_day_numbers = tf.cast(tf.range(tf.shape(y_true)[1], 0, -1), dtype=tf.float64)
    reverse_day_numbers = tf.reshape(reverse_day_numbers, (1, -1, 1))

    profit = tf.where(
        (y_pred_cum > 0) & (y_true_cum > 0), 
        y_true_cum, 
        profit
    )
    profit = tf.where(
        (y_pred_cum > 0) & (y_true_cum < 0),
        -y_true_cum,
        profit
    )
    profit = tf.where(
        (y_pred_cum < 0) & (y_true_cum > 0),
        -y_true_cum,
        profit
    )
    profit = tf.where(
        (y_pred_cum < 0) & (y_true_cum < 0),
        y_true_cum,
        profit
    )

    print("cumulative profit loss ", profit)

    avg_cumulative_profit_per_day = profit / day_numbers


    profit_loss = -avg_cumulative_profit_per_day * profit_multiplier
    print("PROFIT_LOSS", profit_loss)
    # Regulate profit/loss by the decision to act
    #
    regulated_loss = decision * profit_loss + 1e-10
    print("REGULATED_LOSS", regulated_loss)

    # Penalty for not acting when there is significant market movement
    y_true_reverse_cum = tf.reverse(tf.cumsum(tf.reverse(y_true, [1]), axis=1), [1])
    print("Y_TRUE_REVERSE_CUM", y_true_reverse_cum)
    y_true_reverse_cum_per_day = y_true_reverse_cum / reverse_day_numbers
    abstain_penalty = financial_penalty * (1 - decision) * tf.abs(y_true_reverse_cum_per_day)  # Apply penalty based on reverse cumulative movements missed
    print("ABSTAIN_PENALTY", abstain_penalty)

    # add small constant to decision so we evaluate the error a little bit 
    decision = decision + 1e-1

    prediction_error = error_weight *tf.abs(y_true - y_pred) # Penalize the model for incorrect predictions

    # Combine into a total loss
    total_loss = tf.reduce_mean(regulated_loss + abstain_penalty + prediction_error)

    return total_loss

In [14]:
custom_loss_updated(y_fake_true, np.concatenate([y_fake, end_decision_fake], axis=-1))

Decision [[[1.        ]
  [0.94736842]
  [0.89473684]
  [0.84210526]
  [0.78947368]
  [0.73684211]
  [0.68421053]
  [0.63157895]
  [0.57894737]
  [0.52631579]
  [0.47368421]
  [0.42105263]
  [0.36842105]
  [0.31578947]
  [0.26315789]
  [0.21052632]
  [0.15789474]
  [0.10526316]
  [0.05263158]
  [0.        ]]]
Y_TRUE_CUM tf.Tensor(
[[[ 0.79945313]
  [ 1.17394612]
  [ 1.65836761]
  [ 2.13946156]
  [ 2.49015103]
  [ 2.80490923]
  [ 3.80128259]
  [ 4.09668756]
  [ 4.84135269]
  [ 5.6784413 ]
  [ 6.33416383]
  [ 6.51843059]
  [ 7.37660336]
  [ 8.32535644]
  [ 9.06273484]
  [ 9.16659014]
  [ 9.51345577]
  [ 9.62712699]
  [10.3301894 ]
  [10.84174768]]], shape=(1, 20, 1), dtype=float64)
Y_PRED_CUM tf.Tensor(
[[[ 0.55526325]
  [ 0.64950912]
  [ 1.12530884]
  [ 1.42109321]
  [ 1.87297601]
  [ 2.53693504]
  [ 3.37589845]
  [ 3.47979106]
  [ 4.29129928]
  [ 4.79763865]
  [ 4.98885685]
  [ 5.9741993 ]
  [ 6.56941075]
  [ 7.41408511]
  [ 8.10630697]
  [ 8.31043338]
  [ 8.39720007]
  [ 9.10786757]
 

<tf.Tensor: shape=(), dtype=float64, numpy=-0.04636034590974777>